In [9]:
# импорт необходимых библиотек
import numpy as np
from plotly.subplots import make_subplots
from tqdm import tqdm
from typing import Union
import plotly.graph_objects as go
from scipy import stats
from math import asin

import scipy.stats as st
from statsmodels.stats.power import tt_ind_solve_power
from statsmodels.stats.power import zt_ind_solve_power
from statsmodels.stats.proportion import proportion_effectsize
from statsmodels.stats import proportion
from statsmodels.stats.meta_analysis import effectsize_smd

## Анализ онлайн - магазина Ozon

На основе выбранных продуктов из первого вебинара, выберите 2 A/B-теста и составьте план эксперимента, чтобы избежать Peeking Problem

Распишите, какие метрики и как вы будете анализировать



## **Tест №1** 

Предположим, что   бесплатная доставка по промокоду увеличит конверсию в покупку на 8%, уменьшит процент брошенных корзин на 3% 

### **Что делаем**

Контрольная версия: Не добавляем бесплатную доставку в описание над корзиной

Тестовая версия: Добавляем  баннер с описанием бесплатной доставки

### **На каких пользователях тестируем**
  
На постоянных покупателях

### **Главные метрики**
  
Конверсия в покупку, процент брошенных корзин

### **Ожидаемый эффект**
  
рост конверсии в покупку на 10%, уменьшение процента брошенных корзин на 3% 

### **План действий:**
  
Если наш эксперимент будет положительным и мы зафиксируем ожидаемое улучшение в в ключевых метриках и не посадим добавочные, то масштабируем изменение и «выкатываем» его на всех пользователей.
Если основные метрики падают или растут недостаточно, либо добавочные метрики падают, откатываем эксперимент. 


## **Перед стартом.**


Предположим, что постоянных покупателей - 1000000 человек

Тест запускаем на 10% от постоянных покупателей - на 10000 пользователях в день.

Допустим, что сейчас у нас конверсия в покупку (%CR) для постоянных пользователей равна 10%, мы предполагаем увеличение на 8% до 10,8%

alpha = 5% и beta = 20%


#### Формулы

In [10]:
def calc_proportion_es(prob1: float, prob2: float):
  return abs(proportion_effectsize(prob1, prob2))


def calc_proportion_es_alt(conv1: float, conv2: float, prob1: float, prob2: float):
  return 2 * asin(np.sqrt(conv1/nobs1)) - 2 * asin(np.sqrt(conv2/nobs2))


def calc_continuous_es(mean_control: Union[float, int],
                       std_control: Union[float, int],
                       mean_test: Union[float, int],
                       std_test: Union[float, int]):

  return abs(effectsize_smd(mean_control,
                            std_control,
                            1e4,
                            mean_test,
                            std_test,
                            1e4)[0])


def calc_continuous_es_alt(mean_control: Union[float, int],
                           std_control: Union[float, int],
                           mean_test: Union[float, int],
                           std_test: Union[float, int]):

  effect_size = (mean_test - mean_control) / \
      ((std_control**2 + std_test**2) / 2) ** 0.5
  return effect_size


def calc_sample_size_continuous(effect_size: float,
                                alpha: float = .05,
                                beta: float = .2,
                                ratio: Union[float, int] = 1):

  n = tt_ind_solve_power(effect_size=effect_size,
                         alpha=alpha,
                         power=(1 - beta),
                         ratio=ratio,
                         )

  return int(n * 2)


def calc_sample_size_proportion(effect_size: float,
                                alpha: float = .05,
                                beta: float = .2,
                                ratio: Union[float, int] = 1):

  n = zt_ind_solve_power(effect_size=effect_size,
                         alpha=alpha,
                         power=(1 - beta),
                         ratio=ratio,
                         )

  return int(n * 2)


In [11]:
# считаем effect_size
bounce_rate_1, bounce_rate_2 = 0.1, 0.108

es_prop = calc_proportion_es(bounce_rate_1, bounce_rate_2)
print(f'effect_size = {es_prop}')


effect_size = 0.02621166016024057


In [12]:
# Рассчитываем размер выборки для проведения теста
print(
    f'Общие кол-во необходимых наблюдений {calc_sample_size_proportion(es_prop)}')


Общие кол-во необходимых наблюдений 45695


In [13]:
# Рассчитываем время для проведения теста
users_average = 10000

print(
    f'Время для проведения теста: {calc_sample_size_proportion(es_prop) / users_average}')


Время для проведения теста: 4.5695


## **Тест  №2** 

Если мы разобъем банер с горячими скидками на 2 блока, то увеличим количество заказов на 2 %

### **Что делаем**

Контрольная версия: Оставляем все без изменений

Тестовая версия: Разбиваем банер на 2 части

### **На каких пользователях тестируем**
  
На постоянных покупателях

### **Главные метрики**
  
Частота заказов, конверсия в покупку, процент брошенных корзин

### **Ожидаемый эффект**
  
рост частоты заказов на 2%  

### **План действий**
  
Если наш эксперимент будет положительным и мы зафиксируем ожидаемое улучшение в в ключевых метриках и не посадим добавочные, то масштабируем изменение и «выкатываем» его на всех пользователей.
Если основные метрики падают или растут недостаточно, либо добавочные метрики падают, откатываем эксперимент. 


Тест запускаем на 10% от постоянных покупателей - на 10000 пользователях в день.

Допустим, что сейчас у нас частота заказов для постоянных пользователей равна 7, мы предполагаем увеличение на 2% до 7,14

alpha = 5% и beta = 20%

In [14]:
# считаем effect_size
bounce_rate_1, bounce_rate_2 = 0.07, 0.0714

es_prop = calc_proportion_es(bounce_rate_1, bounce_rate_2)
print(f'effect_size = {es_prop}')


effect_size = 0.005461921360775279


In [15]:
# Рассчитываем размер выборки для проведения теста
print(
    f'Общие кол-во необходимых наблюдений {calc_sample_size_proportion(es_prop)}')


Общие кол-во необходимых наблюдений 1052387


In [16]:
# Рассчитываем время для проведения теста
users_average = 10000

print(
    f'Время для проведения теста: {calc_sample_size_proportion(es_prop) / users_average}')


Время для проведения теста: 105.2387
